In [1]:
import pandas as pd
from pycaret.clustering import setup, create_model, assign_model, plot_model, save_model, load_model, predict_model
import json
from openai import OpenAI
from dotenv import dotenv_values
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, Distance, VectorParams
import streamlit as st

In [2]:
env=dotenv_values('.env')

openai_client= OpenAI(api_key=env['OPENAI_API_KEY'])

#openai_client= OpenAI(api_key=env['api_key'])

In [3]:
df=pd.read_csv('welcome_survey_simple_v2.csv', sep=";")

In [4]:
kmeans_pipeline=load_model('clustering_pipeline_v2')

Transformation Pipeline and Model Successfully Loaded


In [5]:
df.head()

,age,edu_level,fav_animals,fav_place,gender
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna
2,45-54,Wyższe,Psy,W lesie,Mężczyzna
3,35-44,Średnie,Koty,W górach,Mężczyzna
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   age          229 non-null    object
 1   edu_level    229 non-null    object
 2   fav_animals  229 non-null    object
 3   fav_place    200 non-null    object
 4   gender       227 non-null    object
dtypes: object(5)
memory usage: 9.1+ KB


In [7]:
len(df)
df

,age,edu_level,fav_animals,fav_place,gender
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna
2,45-54,Wyższe,Psy,W lesie,Mężczyzna
3,35-44,Średnie,Koty,W górach,Mężczyzna
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna
...,...,...,...,...,...
224,35-44,Wyższe,Koty,Inne,Kobieta
225,45-54,Wyższe,Inne,W lesie,Mężczyzna
226,25-34,Wyższe,Psy,W górach,Mężczyzna
227,35-44,Wyższe,Brak ulubionych,W górach,Mężczyzna


In [8]:
s=setup(df, session_id=123)
s

,Description,Value
0,Session id,123
1,Original data shape,"(229, 5)"
2,Transformed data shape,"(229, 21)"
3,Categorical features,5
4,Rows with missing values,13.1%
5,Preprocess,True
6,Imputation type,simple
7,Numeric imputation,mean
8,Categorical imputation,mode
9,Maximum one-hot encoding,-1


In [9]:
s.dataset.head()

,age,edu_level,fav_animals,fav_place,gender
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna
2,45-54,Wyższe,Psy,W lesie,Mężczyzna
3,35-44,Średnie,Koty,W górach,Mężczyzna
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna


In [10]:
kmeans=create_model('kmeans', num_clusters=12)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.2148,21.6655,1.7164,0,0,0


In [11]:
df_with_clusters= assign_model(kmeans)
df_with_clusters

,age,edu_level,fav_animals,fav_place,gender,Cluster
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta,Cluster 7
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna,Cluster 10
2,45-54,Wyższe,Psy,W lesie,Mężczyzna,Cluster 3
3,35-44,Średnie,Koty,W górach,Mężczyzna,Cluster 4
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna,Cluster 0
...,...,...,...,...,...,...
224,35-44,Wyższe,Koty,Inne,Kobieta,Cluster 2
225,45-54,Wyższe,Inne,W lesie,Mężczyzna,Cluster 3
226,25-34,Wyższe,Psy,W górach,Mężczyzna,Cluster 8
227,35-44,Wyższe,Brak ulubionych,W górach,Mężczyzna,Cluster 2


In [12]:

df_with_clusters['Cluster'].value_counts()

Cluster
Cluster 0     36
Cluster 3     25
Cluster 8     25
Cluster 1     24
Cluster 6     24
Cluster 2     20
Cluster 5     20
Cluster 4     16
Cluster 10    14
Cluster 11    10
Cluster 7      8
Cluster 9      7
Name: count, dtype: int64

In [13]:
save_model(kmeans,'clustering_pipeline_v2.pkl', verbose=False)

(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=[], transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['age', 'edu_level', 'fav_animals',
                                              'fav_place', 'gender'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('ordinal_encoding',
                  TransformerWrapper(include=['gender'],
                                     transfo...
                                                                          'data_type': dtype('O'),
                                                                          'mapping': Kobieta      0
 Mężczyzna    1
 NaN         -1
 dtype: int64}]))),
                 ('onehot_encoding',
                  TransformerWrapper(include=['age', 'edu_level', 'fav_animals',
                                             

In [14]:
#kmeans_pipeline=load_model('clustering_pipeline_v2')
kmeans_pipeline

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=[], transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['age', 'edu_level', 'fav_animals',
                                             'fav_place', 'gender'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('ordinal_encoding',
                 TransformerWrapper(include=['gender'],
                                    transfo...
                                                                         'data_type': dtype('O'),
                                                                         'mapping': Kobieta      0
Mężczyzna    1
NaN         -1
dtype: int64}]))),
                ('onehot_encoding',
                 TransformerWrapper(include=['age', 'edu_level', 'fav_animals',
                                             'fav_place'],
                                    transformer=OneHotEncoder(cols=['age',
                                                                    'edu_level',
                                                                    'fav_animals',
                                                                    'fav_place'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('trained_model', KMeans(n_clusters=12, random_state=123))])

In [16]:
df_with_clusters=predict_model(model=kmeans_pipeline, data=df)

In [17]:
df_with_clusters['Cluster'].value_counts()

Cluster
Cluster 0     36
Cluster 3     25
Cluster 8     25
Cluster 1     24
Cluster 6     24
Cluster 2     20
Cluster 5     20
Cluster 4     16
Cluster 10    14
Cluster 11    10
Cluster 7      8
Cluster 9      7
Name: count, dtype: int64

In [18]:
cluster_descriptions={}
for cluster_id in df_with_clusters['Cluster'].unique():
    cluster_df= df_with_clusters[df_with_clusters['Cluster']==cluster_id]
    summary=""
    for column in df_with_clusters:
        if column=='Cluster':
            continue

        value_counts=cluster_df[column].value_counts()
        value_counts_str=','.join([f"{idx}: {cnt}" for idx, cnt in value_counts.items()])
        summary+=f"{column}- {value_counts_str}\n"
    cluster_descriptions[cluster_id]=summary


In [19]:
print(cluster_descriptions["Cluster 11"])

age- 25-34: 8,45-54: 2,18-24: 0,35-44: 0,55-64: 0,<18: 0,>=65: 0,unknown: 0
edu_level- Wyższe: 10,Podstawowe: 0,Średnie: 0
fav_animals- Koty: 8,Brak ulubionych: 1,Inne: 1,Koty i Psy: 0,Psy: 0
fav_place- W górach: 7,Nad wodą: 1,Inne: 0,W lesie: 0
gender- Mężczyzna: 9,Kobieta: 1



In [20]:
prompt="Użyliśmu algorytmu klastrowania"
for cluster_id, descriptions in cluster_descriptions.items():
    prompt+= f"\nKlaster{cluster_id}:\n{descriptions}"

prompt+="""
Wygeneruj najlepsze możliwe opisy dla każdego z klasterów. Proszę wygenerować najlepsze nazwy dla każdego z klastrów oraz ich opisy w formacie JSON.

Uzyje formatu JSON.Przykładowo:
{
    "Cluster 0": {
        "name": "Klaster 0",
        "description": "W tym klastrze znajdują się osoby, które..."
    },
    "Cluster 1": {
        "name": "Klaster 1",
        "description": "W tym klastrze znajdują się osoby, które..."
    }
"""
print(prompt)

Użyliśmu algorytmu klastrowania
KlasterCluster 7:
age- 35-44: 6,45-54: 1,<18: 1,18-24: 0,25-34: 0,55-64: 0,>=65: 0,unknown: 0
edu_level- Średnie: 7,Podstawowe: 1,Wyższe: 0
fav_animals- Psy: 3,Brak ulubionych: 2,Koty: 2,Inne: 1,Koty i Psy: 0
fav_place- Nad wodą: 4,W lesie: 1,Inne: 0,W górach: 0
gender- Kobieta: 6,Mężczyzna: 2

KlasterCluster 10:
age- 45-54: 5,18-24: 4,25-34: 2,35-44: 2,>=65: 1,55-64: 0,<18: 0,unknown: 0
edu_level- Średnie: 14,Podstawowe: 0,Wyższe: 0
fav_animals- Psy: 14,Brak ulubionych: 0,Inne: 0,Koty: 0,Koty i Psy: 0
fav_place- Nad wodą: 8,W górach: 4,W lesie: 1,Inne: 0
gender- Mężczyzna: 14,Kobieta: 0

KlasterCluster 3:
age- 45-54: 18,55-64: 4,35-44: 2,>=65: 1,18-24: 0,25-34: 0,<18: 0,unknown: 0
edu_level- Wyższe: 25,Podstawowe: 0,Średnie: 0
fav_animals- Psy: 12,Brak ulubionych: 5,Koty: 5,Inne: 3,Koty i Psy: 0
fav_place- W lesie: 25,Inne: 0,Nad wodą: 0,W górach: 0
gender- Mężczyzna: 19,Kobieta: 6

KlasterCluster 4:
age- 25-34: 5,35-44: 4,45-54: 4,18-24: 3,55-64: 0,<18

In [21]:
response=openai_client.chat.completions.create(
    model='gpt-4o',
    temperature=0,
    messages=[
        {
            'role':'user',
            'content':[{'type':'text', 'text':prompt}]
        }
    ]
    
)


In [22]:
result = response.choices[0].message.content.replace("```json", "").replace("```", "").strip()
cluster_names_and_descriptions = json.loads(result)

In [24]:
with open("welcome_survey_cluster_names_and_descriptions_v1.json", "w") as f:
    f.write(json.dumps(cluster_names_and_descriptions))

In [25]:
with open('welcome_survey_cluster_names_and_descriptions_v1.json','r') as f:
    print(json.loads(f.read()))

{'Cluster 0': {'name': 'Młodzi profesjonaliści nad wodą', 'description': 'W tym klastrze znajdują się osoby w wieku 35-44 lat, z wyższym wykształceniem, które preferują spędzanie czasu nad wodą. Większość z nich to mężczyźni, a ich ulubionymi zwierzętami są psy.'}, 'Cluster 1': {'name': 'Doświadczeni górscy entuzjaści', 'description': 'W tym klastrze znajdują się osoby w wieku 45-54 lat, z wyższym wykształceniem, które preferują spędzanie czasu w górach. Większość z nich to mężczyźni, a ich ulubionymi zwierzętami są psy.'}, 'Cluster 2': {'name': 'Miłośnicy kotów i gór', 'description': 'W tym klastrze znajdują się osoby w wieku 35-44 lat, z wyższym wykształceniem, które preferują spędzanie czasu w górach. Wśród ulubionych zwierząt dominują koty, a większość z nich to mężczyźni.'}, 'Cluster 3': {'name': 'Wykształceni leśni odkrywcy', 'description': 'W tym klastrze znajdują się osoby w wieku 45-54 lat, z wyższym wykształceniem, które preferują spędzanie czasu w lesie. Większość z nich to 

In [26]:
st.write("Predicted cluster ID:", predicted_cluster_id)

NameError: name 'predicted_cluster_id' is not defined